# Investigate rollouts and resulting datasets

## Round table

### Visualize rollouts

In [6]:
from pathlib import Path
from src.visualization.render_mp4 import (
    mp4_from_pickle_jupyter,
    unpickle_data,
    pickle_data,
)
from src.common.files import get_raw_paths
import random
from tqdm import tqdm

In [25]:
paths = get_raw_paths(
    environment="sim",
    demo_source="rollout",
    demo_outcome="success",
    task="square_table",
    randomness="low",
)

random.shuffle(paths)

len(paths), paths[:3]

Found the following paths:
    /data/scratch-oc40/pulkitag/ankile/furniture-data/raw/sim/square_table/rollout/low/success/*.pkl*


(1,
 [PosixPath('/data/scratch-oc40/pulkitag/ankile/furniture-data/raw/sim/square_table/rollout/low/success/2024-02-21T15:41:17.pkl')])

In [26]:
for p in tqdm(paths):
    path = Path(p)
    try:
        data = unpickle_data(path)
    except EOFError:
        print(f"Failed to load {path}")

100%|██████████| 1/1 [00:15<00:00, 15.93s/it]


In [27]:
for path in paths[:10]:
    mp4_from_pickle_jupyter(path, filename=path.stem + ".mp4", fps=20)

File saved as 2024-02-21T15:41:17.mp4


In [21]:
paths = get_raw_paths(
    environment="sim",
    demo_source="rollout",
    demo_outcome="success",
    task="lamp",
    randomness="low",
)

# random.shuffle(paths)

paths = sorted(paths, reverse=False)

len(paths), paths[:3]

Found the following paths:
    /data/scratch-oc40/pulkitag/ankile/furniture-data/raw/sim/lamp/rollout/low/success/*.pkl*


(50,
 [PosixPath('/data/scratch-oc40/pulkitag/ankile/furniture-data/raw/sim/lamp/rollout/low/success/2024-02-21T11:59:31.pkl'),
  PosixPath('/data/scratch-oc40/pulkitag/ankile/furniture-data/raw/sim/lamp/rollout/low/success/2024-02-21T12:04:00.pkl'),
  PosixPath('/data/scratch-oc40/pulkitag/ankile/furniture-data/raw/sim/lamp/rollout/low/success/2024-02-21T12:12:26.pkl')])

In [ ]:
for p in tqdm(paths):
    path = Path(p)
    try:
        data = unpickle_data(path)
    except EOFError:
        print(f"Failed to load {path}")

100%|██████████| 1/1 [00:15<00:00, 15.93s/it]


In [22]:
for path in paths[:10]:
    mp4_from_pickle_jupyter(path, filename=path.stem + ".mp4", fps=20)

File saved as 2024-02-21T11:59:31.mp4


File saved as 2024-02-21T12:04:00.mp4


File saved as 2024-02-21T12:12:26.mp4


File saved as 2024-02-21T12:18:28.mp4


File saved as 2024-02-21T12:19:07.mp4


File saved as 2024-02-21T12:24:58.mp4


File saved as 2024-02-21T12:34:12.mp4


File saved as 2024-02-21T12:34:23.mp4


KeyboardInterrupt: 

TMP

In [2]:
from wandb import Api

api = Api()

In [3]:
runs = [
    run for run in api.sweep("sweeps/sweeps/ikfmvatf").runs if run.state == "finished"
]

runs

[<Run robot-rearrangement/sweeps/5i7sdf47 (finished)>,
 <Run robot-rearrangement/sweeps/wuk56v2m (finished)>,
 <Run robot-rearrangement/sweeps/ejwzdvkj (finished)>,
 <Run robot-rearrangement/sweeps/w3ffcse8 (finished)>,
 <Run robot-rearrangement/sweeps/ln9ojbe1 (finished)>,
 <Run robot-rearrangement/sweeps/ss9slx1f (finished)>,
 <Run robot-rearrangement/sweeps/tuyqtrl9 (finished)>]

In [16]:
for run in runs:
    # Set the stats to zero
    run.summary["success_rate"] = 0
    run.summary["average_return"] = 0
    run.summary["total_return"] = 0
    run.summary["average_reward"] = 0
    run.summary["total_reward"] = 0
    run.summary["n_rollouts"] = 0
    run.summary["n_success"] = 0
    run.summary["currently_evaluating"] = False

    # Save the changes
    run.update()